<a href="https://colab.research.google.com/github/zyfer416/Construction-analysis/blob/main/NEW_STAGE_CLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Custom loader to fix RGBA warning
def pil_loader(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

# Transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}


In [27]:
data_dir = '/content/drive/MyDrive/CONSTRUCTION DATASET/Construction_Stages'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform=data_transforms[x],
                                          loader=pil_loader)
                  for x in ['train', 'val']}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=2)
               for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(f"Classes: {class_names}")


Classes: ['Brickwork', 'Foundation', 'Framework', 'Painting', 'Plastering']


In [ ]:
# Check for GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load VGG16 model with pretrained weights
weights = models.VGG16_Weights.DEFAULT
model = models.vgg16(weights=weights)

# Freeze feature extractor parameters
for param in model.features.parameters():
    param.requires_grad = False

# Modify the classifier to match the number of classes
num_classes = len(class_names)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# Move model to the appropriate device
model = model.to(device)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:04<00:00, 128MB/s]


In [ ]:
# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer to update only the classifier parameters
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0001)


In [ ]:
# Number of epochs
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Backward pass and optimize only in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print('Training complete!')


Epoch 1/10
----------


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.6392 Acc: 0.7553
val Loss: 0.7229 Acc: 0.7330
Epoch 2/10
----------


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.1815 Acc: 0.9329
val Loss: 0.4719 Acc: 0.8100
Epoch 3/10
----------


...

train Loss: 0.0190 Acc: 0.9942
val Loss: 0.6636 Acc: 0.8281
Epoch 9/10
----------


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.0039 Acc: 0.9981
val Loss: 0.8733 Acc: 0.7919
Epoch 10/10
----------


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 0.0036 Acc: 0.9981
val Loss: 1.0046 Acc: 0.7919
Training complete!


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/CONSTRUCTION DATASET/vgg16_multiclass.pth')

In [ ]:
from google.colab import files
from PIL import Image
import io

# Reload the model for prediction
model = models.vgg16(weights=None)  # No pretrained weights
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model.load_state_dict(torch.load('/content/drive/MyDrive/CONSTRUCTION DATASET/vgg16_multiclass.pth', map_location=device))
model = model.to(device)
model.eval()

# Define transformation for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Function to predict the class of an image
def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()[0]

    print("\n🔍 Raw prediction scores:")
    for i, prob in enumerate(probs):
        print(f"   {class_names[i]}: {prob:.4f}")

    max_idx = probs.argmax()
    print(f"\n✅ Predicted Stage: {class_names[max_idx]}")

# --- Replace Example usage with Uploading Part ---

# Upload image interactively
uploaded = files.upload()

# Predict for each uploaded image
for image_name in uploaded.keys():
    print(f"\n📂 Uploaded File: {image_name}")
    predict_image(image_name)



Saving IMGGGG.jpg to IMGGGG.jpg

📂 Uploaded File: IMGGGG.jpg

🔍 Raw prediction scores:
   Brickwork: 0.0026
   Foundation: 0.0000
   Framework: 0.0000
   Painting: 0.0000
   Plastering: 0.9974

✅ Predicted Stage: Plastering


In [ ]:
import torch
!pip install pyngrok
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from flask import Flask, request, render_template, jsonify
from PIL import Image
import os

app = Flask(__name__)

# Load Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 5  # Update if different
class_names = ["Brickwork", "Foundation", "Framework", "Painting", "Plastering"]

model = models.vgg16(weights=None)  # No pretrained weights
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model.load_state_dict(torch.load('/content/drive/MyDrive/CONSTRUCTION DATASET/vgg16_multiclass.pth', map_location=device))
model = model.to(device)
model.eval()

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Function to make predictions
def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()[0]

    # Get prediction results
    predictions = {class_names[i]: float(probs[i]) for i in range(len(class_names))}
    predicted_stage = class_names[probs.argmax()]

    return predicted_stage, predictions

# Route for home page
@app.route('/')
def index():
    return render_template('index.html')

# Route for handling image upload
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({"error": "No file uploaded!"}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No file selected!"}), 400

    filepath = os.path.join("uploads", file.filename)
    file.save(filepath)

    predicted_stage, predictions = predict_image(filepath)

    return jsonify({"predicted_stage": predicted_stage, "scores": predictions})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# Paste your token here👇
ngrok.set_auth_token("2vR0ugnAj6bI6pXGpqtbXQP8qGa_6F31ZsEGQFq782HjZFX43")


In [ ]:
from flask import Flask, request
from threading import Thread
from pyngrok import ngrok
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image, UnidentifiedImageError
import os
import tempfile
import subprocess

# --- MODEL LOADING ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class_names = ['Brickwork', 'Foundation', 'Framework', 'Painting', 'Plastering']
num_classes = len(class_names)

model = models.vgg16(weights=None)  # No pretrained weights
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model.load_state_dict(torch.load('/content/drive/MyDrive/CONSTRUCTION DATASET/vgg16_multiclass.pth', map_location=device))
model = model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# --- FLASK APP ---
app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def upload_predict():
    if request.method == "POST":
        file = request.files.get("file")
        if file:
            try:
                with tempfile.NamedTemporaryFile(delete=False, suffix='.jpg') as temp_file:
                    temp_filename = temp_file.name
                    file.save(temp_filename)

                image = Image.open(temp_filename).convert('RGB')
                image = transform(image).unsqueeze(0).to(device)

                with torch.no_grad():
                    outputs = model(image)
                    probs = torch.softmax(outputs, dim=1).cpu().numpy()[0]
                max_idx = probs.argmax()
                predicted_stage = class_names[max_idx]
                raw_scores = {class_names[i]: round(float(probs[i]), 4) for i in range(len(class_names))}

                os.remove(temp_filename)

                return f'''
                <!doctype html>
                <html lang="en">
                <head>
                    <meta charset="utf-8">
                    <title>Prediction Result</title>
                    <style>
                        body {{ text-align: center; padding-top: 50px; font-family: Arial; }}
                        .container {{ display: inline-block; padding: 20px; background: #f4f4f4; border-radius: 10px; }}
                        h2 {{ color: green; }}
                        p {{ font-weight: bold; }}
                    </style>
                </head>
                <body>
                    <div class="container">
                        <h2>Prediction Done Successfully!</h2>
                        <p><strong>Predicted Stage:</strong> {predicted_stage}</p>
                        <h4>Raw Prediction Scores:</h4>
                        {'<br>'.join([f"{k}: {v}" for k, v in raw_scores.items()])}
                        <br><br><a href="/">Upload Another Image</a>
                    </div>
                </body>
                </html>
                '''
            except UnidentifiedImageError:
                return "<h3 style='color:red;'>Error: Unsupported image format. Please upload a valid image.</h3>"

    return '''
    <!doctype html>
    <html lang="en">
    <head>
        <meta charset="utf-8">
        <title>Construction Stage Classifier</title>
        <style>
            body {{ text-align: center; padding-top: 100px; background-color: #f2f2f2; font-family: Arial, sans-serif; }}
            .upload-container {{ display: inline-block; background-color: white; padding: 40px; border-radius: 10px; box-shadow: 0px 0px 10px rgba(0,0,0,0.1); }}
            h1 {{ color: #333; }}
            input[type=file] {{ margin: 20px 0; }}
            button {{
                padding: 10px 20px;
                background-color: #4CAF50;
                color: white;
                border: none;
                border-radius: 5px;
                font-size: 16px;
            }}
            button:hover {{ background-color: #45a049; }}
        </style>
    </head>
    <body>
        <div class="upload-container">
            <h1>Construction Stage Classifier</h1>
            <form method="POST" enctype="multipart/form-data">
                <input type="file" name="file" required><br>
                <button type="submit">Predict Stage</button>
            </form>
        </div>
    </body>
    </html>
    '''

# --- Run Flask app in background ---
def run_flask():
    app.run(port=5000, use_reloader=False)
Thread(target=run_flask).start()

# --- Ngrok connection ---
subprocess.run(["pkill", "-f", "ngrok"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
public_url = ngrok.connect(5000).public_url
print(f"🚀 Public URL: {public_url}")


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


🚀 Public URL: NgrokTunnel: "https://2987-34-168-237-86.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
from flask import Flask, render_template, request, redirect, url_for, session
from threading import Thread
from pyngrok import ngrok
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image, UnidentifiedImageError
import os
import secrets

# Flask App Initialization
app = Flask(__name__)
app.secret_key = secrets.token_hex(16)  # Secret key for session management

# --- MODEL LOADING ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class_names = ['Brickwork', 'Foundation', 'Framework', 'Painting', 'Plastering']
num_classes = len(class_names)

model = models.vgg16(weights=None)  # No pretrained weights
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model.load_state_dict(torch.load('/content/drive/MyDrive/CONSTRUCTION DATASET/vgg16_multiclass.pth', map_location=device))
model = model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dummy User Data (Replace with Database Later)
users = {}

# --- ROUTES ---
@app.route('/')
def home():
    if 'user' not in session:
        return redirect(url_for('login'))
    return render_template('upload.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        if username in users and users[username] == password:
            session['user'] = username
            return redirect(url_for('home'))
        return "Invalid Credentials! Try Again."
    return render_template('login.html')

@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        if username in users:
            return "User already exists!"
        users[username] = password
        return redirect(url_for('login'))
    return render_template('signup.html')

@app.route('/logout')
def logout():
    session.pop('user', None)
    return redirect(url_for('login'))

@app.route('/predict', methods=['POST'])
def predict():
    if 'user' not in session:
        return redirect(url_for('login'))

    file = request.files.get("file")
    if file:
        filename = file.filename
        file.save(filename)
        try:
            image = Image.open(filename).convert('RGB')
            image = transform(image).unsqueeze(0).to(device)
            with torch.no_grad():
                outputs = model(image)
                probs = torch.softmax(outputs, dim=1).cpu().numpy()[0]
            max_idx = probs.argmax()
            predicted_stage = class_names[max_idx]
            raw_scores = {class_names[i]: round(float(probs[i]), 4) for i in range(len(class_names))}
            os.remove(filename)
            return render_template('result.html', predicted_stage=predicted_stage, raw_scores=raw_scores)
        except UnidentifiedImageError:
            return "Error: Unsupported image format. Please upload a valid image."
    return redirect(url_for('home'))

# --- Run Flask app in background ---
def run():
    app.run()
Thread(target=run).start()

# --- Ngrok connection ---
ngrok.kill()  # Kill old tunnels
public_url = ngrok.connect(5000)
print(f"\U0001F680 Public URL: {public_url}")


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


🚀 Public URL: NgrokTunnel: "https://382d-34-168-237-86.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# Paste your token here👇
ngrok.set_auth_token("2ulqdKrogU9XaM4sjrLbnXUqKmN_4C482eHJf2W6g8mt8qA3h")


In [ ]:
from flask import Flask, request
from threading import Thread
from pyngrok import ngrok
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import os

# --- MODEL LOADING SAME AS YOUR COLAB CODE ---

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Class names
class_names = ['Brickwork', 'Foundation', 'Framework', 'Painting', 'Plastering']
num_classes = len(class_names)

# Load model
model = models.vgg16(weights=None)  # No pretrained weights
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model.load_state_dict(torch.load('/content/drive/MyDrive/CONSTRUCTION DATASET/vgg16_multiclass.pth', map_location=device))
model = model.to(device)
model.eval()

# Transform (same as training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# --- FLASK APP ---

app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def upload_predict():
    if request.method == "POST":
        file = request.files.get("file")
        if file:
            filename = file.filename
            file.save(filename)

            # Preprocess image
            image = Image.open(filename).convert('RGB')
            image = transform(image).unsqueeze(0).to(device)

            # Predict
            with torch.no_grad():
                outputs = model(image)
                probs = torch.softmax(outputs, dim=1).cpu().numpy()[0]
            max_idx = probs.argmax()
            predicted_stage = class_names[max_idx]
            raw_scores = {class_names[i]: round(float(probs[i]), 4) for i in range(len(class_names))}

            # Remove file after prediction
            os.remove(filename)

            # Return result page
            return f'''
            <!doctype html>
            <html lang="en">
            <head>
                <meta charset="utf-8">
                <title>Prediction Result</title>
                <style>
                    body {{ text-align: center; padding-top: 50px; font-family: Arial; }}
                    .container {{ display: inline-block; padding: 20px; background: #f4f4f4; border-radius: 10px; }}
                    h2 {{ color: green; }}
                    p {{ font-weight: bold; }}
                </style>
            </head>
            <body>
                <div class="container">
                    <h2>Prediction Done Successfully!</h2>
                    <p><strong>File Uploaded:</strong> {filename}</p>
                    <p><strong>Predicted Stage:</strong> {predicted_stage}</p>
                    <h4>Raw Prediction Scores:</h4>
                    {"<br>".join([f"{k}: {v}" for k, v in raw_scores.items()])}
                    <br><br><a href="/">Upload Another Image</a>
                </div>
            </body>
            </html>
            '''

    # --- Upload page ---
    return '''
<!doctype html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <title>Construction Stage Classifier</title>
    <style>
        body {
            text-align: center;
            padding-top: 100px;
            background-color: #f2f2f2;
            font-family: Arial, sans-serif;
        }
        .upload-container {
            display: inline-block;
            background-color: white;
            padding: 40px;
            border-radius: 10px;
            box-shadow: 0px 0px 10px rgba(0,0,0,0.1);
        }
        h1 {
            color: #333;
        }
        input[type=file] {
            margin: 20px 0;
        }
        button {
            padding: 10px 20px;
            background-color: #4CAF50;
            color: white;
            border: none;
            border-radius: 5px;
            font-size: 16px;
        }
        button:hover {
            background-color: #45a049;
        }
    </style>
</head>
<body>
    <div class="upload-container">
        <h1>Construction Stage Classifier</h1>
        <form method="POST" enctype="multipart/form-data">
            <input type="file" name="file" required><br>
            <button type="submit">Predict Stage</button>
        </form>
    </div>
</body>
</html>
'''

# --- Run Flask app in background ---
def run():
    app.run()

Thread(target=run).start()

# --- Ngrok connection ---
!pkill -f ngrok  # Kill old tunnels
public_url = ngrok.connect(5000)
print(f"🚀 Public URL: {public_url}")


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


🚀 Public URL: NgrokTunnel: "https://15fb-34-168-237-86.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


🚀 Public URL: NgrokTunnel: "https://27f2-34-168-237-86.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
# --- YOLOv8 SETUP ---
from ultralytics import YOLO

# Load YOLOv8 model (you can use yolov8n.pt, yolov8s.pt, etc.)
yolo_model = YOLO("yolov8n.pt")  # using the small model for faster inference



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 81.0MB/s]


In [ ]:
# --- YOLOv8 Detection ---
yolo_results = yolo_model.predict(source=filename, conf=0.4, save=True, save_txt=False)
yolo_output_dir = yolo_results[0].save_dir  # path like runs/detect/predict
yolo_output_img = os.path.join(yolo_output_dir, os.path.basename(filename))

# Copy YOLO image to working dir so Flask can serve it
output_img_path = "detected_" + filename
os.rename(yolo_output_img, output_img_path)



NameError: name 'filename' is not defined

In [ ]:
from flask import Flask, request, send_from_directory
from threading import Thread
from pyngrok import ngrok
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from ultralytics import YOLO

# --- Device Setup ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- Class Names for VGG16 ---
class_names = ['Brickwork', 'Foundation', 'Framework', 'Painting', 'Plastering']
num_classes = len(class_names)

# --- Load VGG16 Model ---
model = models.vgg16(weights=None)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model.load_state_dict(torch.load('/content/drive/MyDrive/CONSTRUCTION DATASET/vgg16_multiclass.pth', map_location=device))
model = model.to(device)
model.eval()

# --- Load YOLOv8 Model ---
yolo_model = YOLO('yolov8n.pt')  # Change to yolov8s.pt or yolov8m.pt if needed

# --- Transform for VGG ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# --- Flask App ---
app = Flask(__name__)
UPLOAD_FOLDER = '/content/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route("/", methods=["GET", "POST"])
def upload_predict():
    if request.method == "POST":
        file = request.files.get("file")
        if file:
            filename = file.filename
            file_path = os.path.join(UPLOAD_FOLDER, filename)
            file.save(file_path)

            # VGG16 Classification
            image = Image.open(file_path).convert('RGB')
            vgg_input = transform(image).unsqueeze(0).to(device)
            with torch.no_grad():
                outputs = model(vgg_input)
                probs = torch.softmax(outputs, dim=1).cpu().numpy()[0]
            max_idx = probs.argmax()
            predicted_stage = class_names[max_idx]
            raw_scores = {class_names[i]: round(float(probs[i]), 4) for i in range(len(class_names))}

            # YOLOv8 Detection
            detection_results = yolo_model.predict(source=file_path, save=True, conf=0.3)
            yolo_save_path = detection_results[0].save_dir
            yolo_output_img_path = os.path.join(yolo_save_path, filename)

            return f'''
            <!doctype html>
            <html lang="en">
            <head>
                <meta charset="utf-8">
                <title>Prediction Result</title>
                <style>
                    body {{ text-align: center; padding-top: 50px; font-family: Arial; }}
                    .container {{ display: inline-block; padding: 20px; background: #f4f4f4; border-radius: 10px; }}
                    h2 {{ color: green; }}
                    p {{ font-weight: bold; }}
                    img {{ max-width: 90%; margin-top: 20px; border: 1px solid #ccc; border-radius: 10px; }}
                </style>
            </head>
            <body>
                <div class="container">
                    <h2>Prediction Completed</h2>
                    <p><strong>File:</strong> {filename}</p>
                    <p><strong>Predicted Stage:</strong> {predicted_stage}</p>
                    <h4>Raw Prediction Scores:</h4>
                    {"<br>".join([f"{k}: {v}" for k, v in raw_scores.items()])}
                    <h4>Detected Objects:</h4>
                    <img src="/files/{filename}" alt="YOLOv8 Detection Output">
                    <br><br><a href="/">🔁 Upload Another Image</a>
                </div>
            </body>
            </html>
            '''

    return '''
    <!doctype html>
    <html lang="en">
    <head>
        <meta charset="utf-8">
        <title>Construction Stage + Object Detection</title>
        <style>
            body {{
                text-align: center;
                padding-top: 100px;
                background-color: #f0f0f0;
                font-family: Arial, sans-serif;
            }}
            .upload-box {{
                display: inline-block;
                background-color: white;
                padding: 40px;
                border-radius: 10px;
                box-shadow: 0px 0px 10px rgba(0,0,0,0.1);
            }}
            h1 {{ color: #333; }}
            input[type=file] {{ margin: 20px 0; }}
            button {{
                padding: 10px 20px;
                background-color: #4CAF50;
                color: white;
                border: none;
                border-radius: 5px;
                font-size: 16px;
            }}
            button:hover {{ background-color: #45a049; }}
        </style>
    </head>
    <body>
        <div class="upload-box">
            <h1>Construction Stage + Object Detector</h1>
            <form method="POST" enctype="multipart/form-data">
                <input type="file" name="file" required><br>
                <button type="submit">Predict</button>
            </form>
        </div>
    </body>
    </html>
    '''

# Serve YOLO output image
@app.route("/files/<filename>")
def send_file(filename):
    yolo_output_dir = yolo_model.predict(source=UPLOAD_FOLDER, save=True, conf=0.3)[0].save_dir
    return send_from_directory(yolo_output_dir, filename)

# Launch Flask in thread
def run():
    app.run()

Thread(target=run).start()

# Kill old tunnels if any, then start Ngrok
!pkill -f ngrok
public_url = ngrok.connect(5000)
print(f"🚀 Public URL: {public_url}")


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


🚀 Public URL: NgrokTunnel: "https://de4e-34-168-237-86.ngrok-free.app" -> "http://localhost:5000"


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/serving.py", line 759, in __init__
    self.server_bind()
  File "/usr/lib/python3.11/http/server.py", line 136, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/usr/lib/python3.11/socketserver.py", line 472, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-16e30b4b66d7>", line 2, in <cell line: 0>
    app.run()
  File "/usr/local/lib/python3.11/dist-packages/flask_ngrok.py", line 88, in new_run
    old_run()
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 662, in run
    run_simple(t.cast(str, host), port, self, **optio

TypeError: object of type 'NoneType' has no len()